# Ground Navigation for Aerial Vehicles

With a satellite reference image, we intend to utilize a SfM solution to match ground segments of images to the reference map.
- Inputs: Reference image (satellite), SfM solution (images, points, cameras), selected images (5)
- Output: Correction solutions for translation, rotation, scaling

Checklist
1. [x] Load in satellite image values 
2. [x] Choose ground segment in each image
3. [x] Load ground segments into a plot
4. [x] Implement an initial guess
5. [x] NEW SSD process on each
6. [ ] Create a correction vector for each
7. [ ] Visualization of each correction vector

Other things to do:
- [ ] More ground plane points
- [ ] Better initial guess

Things to make faster 
- [x] grab_image_pts
- [x] ssd_nxn
- [ ] Further efficiency for ssd??

In [1]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
import imageio
# %matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.path import Path

from groundNAV_agent import *
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# NOTES: 
# OS in JUMBOS: IMG_9446; ID = 4
# 40yd (far hash): IMG_9520; ID = 78
# Jumbo logo: IMG_9531; ID = 89
# 30yd (near hash): IMG_9542; ID = 100
# U in JUMBOS: IMG_9576; ID = 134

In [3]:
# Load in necessary parameters for gNAV agent 
# Define Class Parameters 

images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# Images selected for local corrections
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG" #ID:4
image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9520.JPEG" #ID:78
image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9531.JPEG" #ID:89
image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9542.JPEG" #ID:100
image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9576.JPEG" #ID:134
# Load in satellite reference image
sat_ref = "TTurf/TurfSat.jpg"
# sat_ref = cv2.imread('TTurf/TurfSat.jpg')

# Organize for agent params
images = [image_1, image_2, image_3, image_4, image_5]

In [4]:
# Create class
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, images, sat_ref)

# Grab raw points and RGB data for scene and reference cloud
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d_c)
ref_pts, ref_rgb = gnav.ref_pts, gnav.ref_rgb

In [5]:
# Use ground plane pts to set reference frame 
# Need gravity and height
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
tform_ref_frame = gnav.set_ref_frame(pts_gnd_idx)
tform_ref_frame_pts = gnav.inv_homog_transform(tform_ref_frame)
# print("\nReference frame transformation\n", tform_ref_frame)

In [6]:
# Transform all points to the new coordinate system 
# Not necessary since we aren't using the cloud, but a good visual check for coord frame
tform_ref_frame_inv = gnav.inv_homog_transform(tform_ref_frame)
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(scene_pts, gnav.origin_w, tform_ref_frame_inv)
# print(origin_ref)
# Transform scene cloud to 2D (also as a visual check)
# Note: 2d projection will look off with z=-1; see TTurf_v2 for cropping method
scene_ref_2d = gnav.proj_2d_scene(scene_pts_ref)
# print(scene_ref_2d)

### Grab ground plane points from each desired image 

In [7]:
# Image 1 - OS in Jumbos
imnum = 0
x,y = 150,1725
side_x = 2250 # WIDTH
side_y = 750 # HEIGHT

# x,y = 150, 1600
# side_x = 2500 # WIDTH
# side_y = 1000 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_loc)
# print(pts_rgb)

In [8]:
# Image 2 - 40yd (far)
imnum = 1
x,y = 1150,1150
side_x = 1100 # WIDTH
side_y = 900 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [9]:
# Image 3 - Jumbo logo
imnum = 2
x,y = 400,1500
side_x = 2000 # WIDTH
side_y = 1000 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [10]:
# Image 4 - 30yd (near)
imnum = 3
x,y = 1350,1450
side_x = 1200 # WIDTH
side_y = 500 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [11]:
# Image 5 - U in Jumbos
imnum = 4
x,y = 200,1900
side_x = 2250 # WIDTH
side_y = 1300 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y, imnum) 

# Get necessary location and rgb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y, imnum)
# print(pts_rgb)

In [12]:
# Generate projection of image sections 
for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    # Unit vectors in world coords 
    pts_vec_c, pts_rgb_gnd = gnav.unit_vec_c(i)
    gnav.im_mosaic[i] = {'rgbc': pts_rgb_gnd}

    # Get transformation matrix that move from camera coords to world coords
    id = gnav.im_ids[i]
    homog_w2c, homog_c2w = gnav.get_pose_id(id)
    # print('Homogeneous transformation from world to camera \n', homog_c2w)
    # print('\n Homogeneous transformation from camera to world \n', homog_w2c)

    # Transform to world coords
    origin_c, pts_loc_w, pts_vec_w = gnav.unit_vec_tform(pts_vec_c, gnav.origin_w, homog_c2w)
    # print('\n New camera frame origin = ', origin_c)
    
    # Get new points 
    ranges, new_pts_w = gnav.pt_range(pts_vec_w, homog_c2w, origin_c)
    # print('\nNew Points \n', new_pts_w)

    # Transfer points to reference frame
    __, new_pts_r, pts_vec_r = gnav.unit_vec_tform(new_pts_w, gnav.origin_w, tform_ref_frame_pts)

    # Convert points to grayscale 
    gray_c = gnav.conv_to_gray(gnav.im_mosaic[i]['rgbc'],i)
    # print(gray_c)

    # Put new points and grayscale colors in image mosaic
    gnav.im_mosaic[i]['pts'] = new_pts_r
    gnav.im_mosaic[i]['color_g'] = gray_c
    
    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4


### Initial Guess

In [13]:
# Implementing an initial guess for the local image 

# SCALE for initial guess 
scale = gnav.focal/43

# Translation and Rotation for initial guess
x = 400
y = 300
yaw = 0

# # Initial guess from v2 (for verification purposes)
# scale = gnav.focal/46
# x = 400
# y = 295
# yaw = 0

tform_guess = gnav.tform_create(x,y,0,0,0,yaw)

for i in range(len(images)):
# Just for the first image for now
# for i in range(1):
    loc_im_pts = gnav.im_mosaic[i]['pts'].copy()
    loc_im_pts[:, :2] *= scale
    # Get new points 
    __, loc_im_pts_guess, loc_im_vec_guess = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_guess)
    gnav.im_pts_best_guess[i] = {'pts': loc_im_pts_guess}
    # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 

    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4


In [14]:
# PLOTTING THE NEW SCENE MOSAIC

# Use open3d to create point cloud visualization 
# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Mosaic scene with satellite reference")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# Create point cloud for image points
im0_cloud = o3d.geometry.PointCloud()
im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0]['pts'])
im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# Create point cloud for image points
im1_cloud = o3d.geometry.PointCloud()
im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[1]['pts'])
im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])

# Create point cloud for image points
im2_cloud = o3d.geometry.PointCloud()
im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[2]['pts'])
im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])

# Create point cloud for image points
im3_cloud = o3d.geometry.PointCloud()
im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[3]['pts'])
im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])

# Create point cloud for image points
im4_cloud = o3d.geometry.PointCloud()
im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[4]['pts'])
im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])

# Create point cloud for reference cloud (satellite)
ref_cloud = o3d.geometry.PointCloud()
ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(im0_cloud)
vis.add_geometry(im1_cloud)
vis.add_geometry(im2_cloud)
vis.add_geometry(im3_cloud)
vis.add_geometry(im4_cloud)
vis.add_geometry(ref_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 67
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 67
pci id for fd 67: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


### SSD Process - NEW VERSION

Steps:
- [x] Get four corners of image pts
- [x] Generate a polygon based on corners
- [x] Grab satellite points within polygon
- [x] Find nearest wedge point
- [x] SSD
- [x] Create functions within agent file

Notes: 
- Number of points in SSD process: 19693
- Divided by 5: 3939 (9800 when / 2)

In [19]:
n = 3
# for i in range(len(images)):
# Just for the first image for now 
for imnum in range(1):
    ssds = gnav.ssd_nxn(n, imnum)
print("Done, SSDs are\n", ssds)

SSD =  332.0432553805921
SSD =  334.43198722219154
SSD =  335.6513734890119
SSD =  335.6969463917417
SSD =  330.7158760957017
SSD =  324.9931361956632
SSD =  326.2393935735948
SSD =  337.4660545193849
SSD =  338.20072376967323
SSD =  338.71696543057294
SSD =  339.5203898365706
SSD =  336.1619376105036
SSD =  327.79717528831986
SSD =  325.13063817182626
SSD =  341.5560408630988
SSD =  337.33645845633225
SSD =  336.4090184255748
SSD =  339.06577745671666
SSD =  338.3459663071588
SSD =  334.66856380043066
SSD =  327.1232198250366
SSD =  340.0300843540639
SSD =  335.5101507592926
SSD =  334.33555966356016
SSD =  331.69594849862364
SSD =  334.9038524125029
SSD =  336.5695259842061
SSD =  331.1336055843599
SSD =  333.124458210273
SSD =  330.2338487831296
SSD =  328.36954117836217
SSD =  327.78556935214147
SSD =  328.2918277758247
SSD =  329.6600026931642
SSD =  328.12083763357174
SSD =  326.3040362802461
SSD =  326.26278036329103
SSD =  326.6992468915494
SSD =  327.9584913975087
SSD =  325.3

### Plotting SSD RESULTS 

In [16]:
# Plot and visualize SSD values 

# Create a 5x5 grid of x and y coordinates
x = np.linspace(-n, n, 2*n+1)
y = np.linspace(-n, n, 2*n+1)
X, Y = np.meshgrid(x, y)
# print(ssds)

# print(x)
# print(X)
# print(Y)


In [17]:
# Create the figure and 3D axis
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D vectors
ax.quiver(X, Y, np.zeros_like(ssds), np.zeros_like(ssds), np.zeros_like(ssds), (ssds/800)**8)

# Set axis limits
ax.set_xlim([-8, 8])  # X axis range
ax.set_ylim([-8, 8])  # Y axis range
ax.set_zlim([0, (np.max(ssds) / 800)**8])  # Z axis range, adjust based on your data

# Labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Pixel Correction Vector Field')

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 73
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 73
pci id for fd 73: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


Text(0.5, 0.92, 'Pixel Correction Vector Field')

In [18]:
# Draw a vector from the maximum to the minimum ssd
idmax_x, idmax_y = np.unravel_index(np.argmax(ssds), ssds.shape)
# max_index = np.unravel_index(np.argmax(matrix), matrix.shape)
idmin_x, idmin_y = np.unravel_index(np.argmin(ssds), ssds.shape)

print(idmax_x, idmax_y)
print(ssds[idmax_x, idmax_y])
print(idmin_x, idmin_y)
print(ssds[idmin_x, idmin_y])

shiftx_max = idmax_x-n
shifty_max = idmax_y-n
shiftx_min = idmin_x-n
shifty_min = idmin_y-n
print(shiftx_max, shifty_max)
print(shiftx_min, shifty_min)

basex = np.mean(gnav.im_pts_best_guess[0][:,0])
basey = np.mean(gnav.im_pts_best_guess[0][:,1])

print(basex, basey)

P1 = np.array([basex+shiftx_max*3, basey+shifty_max*3, -5])
P2 = np.array([basex+shiftx_min*3, basey+shifty_min*3, -5])
# CHANGE VECTOR
points_n = [P1,P2]
lines_n = []
lines_n.append([0,1])

1 0
174.30330507621684
5 5
155.64923405737795
-2 -3
2 2


TypeError: unhashable type: 'slice'

In [ ]:
# 

# Plotting Tools

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE ORIGINAL ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING the SATELLITE REFERENCE image as a cloud

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=500)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(ref_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE REFERENCE ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts_ref)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE REFERENCE ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="2D PROJECTION scene with REFERENCE ORIGIN")

# # Create axes @ origin
# # axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_ref_2d)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# # vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE NEW SCENE MOSAIC BEFORE INITIAL GUESS

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene with satellite reference")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for image points
# im0_cloud = o3d.geometry.PointCloud()
# im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['pts'])
# im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# # Create point cloud for image points
# im1_cloud = o3d.geometry.PointCloud()
# im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['pts'])
# im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])

# # Create point cloud for image points
# im2_cloud = o3d.geometry.PointCloud()
# im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['pts'])
# im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])

# # Create point cloud for image points
# im3_cloud = o3d.geometry.PointCloud()
# im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['pts'])
# im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])

# # Create point cloud for image points
# im4_cloud = o3d.geometry.PointCloud()
# im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['pts'])
# im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(im0_cloud)
# vis.add_geometry(im1_cloud)
# vis.add_geometry(im2_cloud)
# vis.add_geometry(im3_cloud)
# vis.add_geometry(im4_cloud)
# vis.add_geometry(ref_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE NEW SCENE MOSAIC

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Mosaic scene with satellite reference")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for image points
# im0_cloud = o3d.geometry.PointCloud()
# im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0])
# im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# # Create lineset for adjustment vector 
# # Convert points and lines to numpy arrays
# points_n = np.array(points_n)
# lines_n = np.array(lines_n)
# # Create LineSet
# line_set = o3d.geometry.LineSet()
# line_set.points = o3d.utility.Vector3dVector(points_n)
# line_set.lines = o3d.utility.Vector2iVector(lines_n)
# line_set.paint_uniform_color([0, 1, 0])  # RGB for red (1, 0, 0)

# # Create point cloud for image points
# im1_cloud = o3d.geometry.PointCloud()
# im1_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[1])
# im1_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[1]['color_g'])

# # Create point cloud for image points
# im2_cloud = o3d.geometry.PointCloud()
# im2_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[2])
# im2_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[2]['color_g'])

# # Create point cloud for image points
# im3_cloud = o3d.geometry.PointCloud()
# im3_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[3])
# im3_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[3]['color_g'])

# # Create point cloud for image points
# im4_cloud = o3d.geometry.PointCloud()
# im4_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[4])
# im4_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[4]['color_g'])

# # Create point cloud for reference cloud (satellite)
# ref_cloud = o3d.geometry.PointCloud()
# ref_cloud.points = o3d.utility.Vector3dVector(gnav.ref_pts)
# ref_cloud.colors = o3d.utility.Vector3dVector(gnav.ref_rgb)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(im0_cloud)
# vis.add_geometry(line_set)
# vis.add_geometry(im1_cloud)
# vis.add_geometry(im2_cloud)
# vis.add_geometry(im3_cloud)
# vis.add_geometry(im4_cloud)
# vis.add_geometry(ref_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE NEW SCENE MOSAIC

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="ROI poly visualization for new SSD (with sat ref)")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for image points
# im0_cloud = o3d.geometry.PointCloud()
# im0_cloud.points = o3d.utility.Vector3dVector(gnav.im_pts_best_guess[0]['pts'])
# im0_cloud.colors = o3d.utility.Vector3dVector(gnav.im_mosaic[0]['color_g'])

# corners_cloud = o3d.geometry.PointCloud()
# corners_cloud.points = o3d.utility.Vector3dVector(points)
# corners_cloud.paint_uniform_color([1, 0, 0])  # RGB for red (1, 0, 0)

# poly_sat = o3d.geometry.PointCloud()
# poly_sat.points = o3d.utility.Vector3dVector(inside_pts)
# poly_sat.paint_uniform_color([0, 0, 1])  # RGB for red (1, 0, 0)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(im0_cloud)
# vis.add_geometry(corners_cloud)
# vis.add_geometry(ref_cloud)
# vis.add_geometry(poly_sat)


# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()